# SPT Model Draft

Allen Pinjic - Created on June 6th, 2022

In [1]:
from astropy.io.fits import getdata
from astropy.table import Table
from astropy.cosmology import WMAP9 as cosmo

In [2]:
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import pylab as plt
import pymc3 as pm
import aesara
import matplotlib.font_manager
import scipy.stats
import scipy.optimize
import math

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
%matplotlib inline

## Retrieving SPT Data

In [4]:
fname = '../data_set/sptecs_catalog_oct919.fits'

data = Table(getdata(fname))

In [5]:
df = data.to_pandas()
h=0.7
df['log_M500'] = np.log(df.M500*1e14*h)
df['log_LAMBDA_CHISQ'] = np.log(df.LAMBDA_CHISQ)

/data0/sw/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
## Calculating the log of the mean mass error ##
m500 = df.M500.to_numpy()*1e14*h
log_m500 = np.log(m500)

lerr = df.M500_LERR.to_numpy()*1e14*h
uerr = df.M500_UERR.to_numpy()*1e14*h

m500_err = (lerr+uerr)/2 # mean mass error
log_m500_err = m500_err/m500

# Eliminating NaN values from the log_m500_err data set
nnan = ~np.isnan(log_m500_err)
nnan2 = ~np.isnan(log_m500)

<ipython-input-6-5c4766f6076c>:3: RuntimeWarning: divide by zero encountered in log
  log_m500 = np.log(m500)
<ipython-input-6-5c4766f6076c>:9: RuntimeWarning: invalid value encountered in true_divide
  log_m500_err = m500_err/m500


In [7]:
## Calculating the log of the richness error ##

lbd = df.LAMBDA_CHISQ.to_numpy()
lbd_err = df.LAMBDA_CHISQ_E.to_numpy()

log_lbd = df.log_LAMBDA_CHISQ
log_lbd_err = lbd_err/lbd

# Eliminating NaN values from the log_lbd_err data set
nnan3 = ~np.isnan(log_lbd_err)
nnan4 = ~np.isnan(log_lbd)

nnan_all = nnan & nnan2 & nnan3 & nnan4

<ipython-input-7-949632b44be5>:7: RuntimeWarning: invalid value encountered in true_divide
  log_lbd_err = lbd_err/lbd


In [8]:
df0 = df
z = df0.REDSHIFT.to_numpy()
lbd = df0.LAMBDA_CHISQ.to_numpy()
zcut = (z > 0.1) & (z < 0.65)
lcut = lbd > 20
final_cut = zcut & lcut

In [9]:
df = df0.loc[final_cut].copy()

## First Draft Version

In [10]:
#Y = np.vstack([x,y])

In [11]:
def model(theta, x, gammasz = 1.0, M0 = 3e14, Ez0=1.):
    M, z  = x
    asz, bsz, csz, albd, blbd, clbd, = theta

    y1 = gammasz*asz + bsz*np.log(M/M0) + csz*np.log(E(z)/Ez0)
    y2 = albd + albd*np.log(M/M0) + clbd*np.log(E(z)/Ez0)
    return np.vstack([y1,y2])
    

In [12]:
from colossus.cosmology import cosmology
from colossus.lss import mass_function
from astropy.io.fits import getdata
from astropy.table import Table
cosmology.setCosmology('WMAP9')

# true = obs lambda
# true SZ signal = obs SZ signal

def joint_probability(y1, y2, mu1, mu2, s1, s2, rho):
    den = 2*np.pi*s1*s2*np,sqrt(1-rho**2)
    num = (y1-mu1)**2/s1**2 + (y2-mu2)**2/s2**2
    num += 2*((y1-mu1)/s1)*((y2-mu2)/s2)/(2*(1-rho**2))
    log_prob = -num/(2*(1-rho**2))
    prob = np.exp(log_prob/den)
    return prob

def prob_lbd_sz(theta, M, z):
    y1, y2 = y
    
    
    # covariance
    s1, s2, rho = theta[-3], theta[-2], theta[-1]
   # cov = np.array([[s1**2, s1*s2*rho], [s1*s2*rho, s2**2]])
   # den - 1./(s1**2*s2**2*(1-rho))
   # cov_inv = np.array([[s1**2, -s1*s2*rho], [-s1*s2*rho, s2**2]])/den
    
    # predictions
    mu1, mu2 = model(theta, x)
    
    #joint probability
    return joint_probability(y1, y2, mu1, mu2, s1, s2, rho)


#comparing predictions with the model

# x = M, z
def hal0_mass_function(M, z):
    return mass_function.massFunction(M, z, mdef = 'vir', model = 'tinker08')


def prob_obs_lambda_integrand(M, z, theta):
    M, z = x
    prob = prob_lbd_sz(theta, x, y)
    hmf = halo_mass_function(M, z)
    integrand = prob*hmf
    return integrand

def prob_obs_lambda(theta, x, y, errors, Norm=1.):
    M, z = x
    value = scipy.integrated.quad(prob_obs_lambda_integrand, 1e13, 1e16, args=(z, theta, y))/Norm
    return value

def log_likelihood(theta, x, y, errors):
    return np.log(prob_obs_lambda(theta, x, y, errors))

## Solving for the Model Directly (via S. Grandis et al. 2021)

In [13]:
## Lisiting constants (or undefined variables for now too) ##

M0 = 3e14

M = log_m500[nnan_all]
# Is M defined as simply the values of the mass or the log of the mass?

z = z[nnan_all]

lbd = log_lbd[nnan_all]
# Is lbd defined as simply the values of lambda or the log of lambda?

lbd_error = log_lbd_err[nnan_all]
# Is lbd_error defined as simply the error values of lambda or the error values of the log of lambda?

chsi = 0
lbd_hat = 0
rho = 0
zeta = 0
scatter_lambda = 0
scatter_zeta = 0
# Temporary values (since they are unknown as of yet)

In [14]:
def E(z):
    # The Hubble constant at the value of z
    Hz = cosmo.H(z).value
    # The Hubble constant at z=0
    H0 = cosmo.H(0).value
    return (Hz/H0)

In [15]:
A_lambda = 76.9
B_lambda = 1.020
C_lambda = 0.29
scatter_lambda = 0.16

#A_lambda = pm.Normal("A_lambda", mu=76.9, sigma=(8.2*3))
#B_lambda = pm.Normal("B_lambda", mu=1.020, sigma=(0.080*3))
#C_lambda = pm.Normal("C_lambda", mu=0.29, sigma=(0.27*3))
#scatter_lambda = pm.HalfCauchy("scatter", 0.16)

A_sze = 5.24
B_sze = 1.534
C_sze = 0.465
scatter_sze = 0.161

#A_lambda = pm.Normal("A_lambda", mu=5.24, sigma=(0.85*3))
#B_lambda = pm.Normal("B_lambda", mu=1.534, sigma=(0.100*3))
#C_lambda = pm.Normal("C_lambda", mu=0.465, sigma=(0.407*3))
#scatter_sze = pm.HalfCauchy("scatter", 0.080)


## Not sure whether to use point values or values on a distribution ##

In [16]:
ln_lbd_given_M = np.log(A_lambda) + (B_lambda)*(np.log(M/M0)) + (C_lambda)*(np.log(E(z)/E(0)))
ln_zeta_given_M = np.log(A_sze) + (B_sze)*(np.log(M/M0)) + (C_sze)*(np.log(E(z)/E(0)))

In [18]:
def log_likelihood_lambda():
    equationPt1 = (1/((-2)*(1 - (rho)**2)))*[((lbd - ln_lbd_given_M)/(scatter_lambda))**2 
                + (-2)*(rho)*((lbd - ln_lbd_given_M)/(scatter_lambda))]
    equationPt2 = (-1/2)*((lbd - lbd_hat)/(lbd_error))**2
    return (equationPt1) + (equationPt2)

def log_likelihood_zeta():
    eq1 = (chsi - math.sqrt((zeta)**2 + 3))**2
    eq2 = ((zeta - ln_zeta_given_M)/(scatter_zeta))**2
    return (eq1 + eq2)
    
def log_likelihood_correlation():
    equation = (-2*rho)*((lbd - ln_lbd_given_M)/(scatter_lambda))*((zeta - ln_zeta_given_M)/(scatter_zeta))
    return equation
    
def log_likelihood_total():
    return (log_likelihood_lambda + log_likelihood_zeta + log_likelihood_correlation + np.log(hal0_mass_function(M, z)))
